In [1]:
import pandas as pd # to read CSV, create locations and measurements from DF
import gmaps # heatmaps and location maps
import requests # request to google places JSON file

# import API key
from config import gkey

In [2]:
# import cities csv as dataframe
city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Vestmannaeyjar,IS,2021-12-04 16:32:25,63.4427,-20.2734,29.39,80,0,9.22
1,1,Mar Del Plata,AR,2021-12-04 16:31:30,-38.0023,-57.5575,60.82,81,75,11.99
2,2,Marathon,US,2021-12-04 16:32:26,44.9000,-89.7668,32.20,62,1,1.01
3,3,Barrow,US,2021-12-04 16:30:02,71.2906,-156.7887,8.62,78,90,14.97
4,4,Nikolskoye,RU,2021-12-04 16:32:26,59.7035,30.7861,19.02,95,100,4.99


In [3]:
# check if any in city_data_df is null
city_data_df[city_data_df.isnull().any(axis=1)]

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
12,12,Oranjemund,NaN,2021-12-04 16:32:28,-28.5500,16.4333,63.32,80,39,9.64
261,261,Luderitz,NaN,2021-12-04 16:36:42,-26.6481,15.1594,64.65,75,2,9.73
341,341,Okakarara,NaN,2021-12-04 16:41:05,-20.5833,17.4333,84.94,18,56,19.10
457,457,Opuwo,NaN,2021-12-04 16:43:39,-18.0607,13.8400,83.80,23,66,17.40
461,461,Henties Bay,NaN,2021-12-04 16:44:40,-22.1160,14.2845,65.86,78,0,15.61
521,521,Eenhana,NaN,2021-12-04 16:45:58,-17.4667,16.3333,79.25,50,90,11.03


In [4]:
# all NaN belong to cities from Namibia, can fill all NaN with NA, Namibia's actual 2 letter code
city_data_df.fillna('NA', inplace=True)
city_data_df.loc[341], city_data_df.loc[521], city_data_df.loc[457]

(City_ID                       341
 City                    Okakarara
 Country                        NA
 Date          2021-12-04 16:41:05
 Lat                      -20.5833
 Lng                       17.4333
 Max Temp                    84.94
 Humidity                       18
 Cloudiness                     56
 Wind Speed                   19.1
 Name: 341, dtype: object,
 City_ID                       521
 City                      Eenhana
 Country                        NA
 Date          2021-12-04 16:45:58
 Lat                      -17.4667
 Lng                       16.3333
 Max Temp                    79.25
 Humidity                       50
 Cloudiness                     90
 Wind Speed                  11.03
 Name: 521, dtype: object,
 City_ID                       457
 City                        Opuwo
 Country                        NA
 Date          2021-12-04 16:43:39
 Lat                      -18.0607
 Lng                         13.84
 Max Temp                     83.8
 

In [5]:
# check dataframe for datatypes to ensure we can use gmaps
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [6]:
# configure gmaps to use API key
gmaps.configure(api_key=gkey)

In [7]:
# heatmap of temperature
# get latitude and longitude into locations
locations = city_data_df[['Lat','Lng']]

# get maximum temperature
max_temp = city_data_df['Max Temp']

# Assign the figure variable
fig = gmaps.figure(zoom_level=1.5, center=(30.0,31.0))

# Use only positive temperatures
#temps = []
#for temp in max_temp:
    #temps.append(max(temp,0))

# Add heatmap variable (list comprehension for weights to define only positive temperatures)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], dissipating=False, max_intensity=400, point_radius=5)

# Add heatmap layer
fig.add_layer(heat_layer)

# Call fig to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# ask customer to specify min and max temperature value
min_temp = float(input('What is the minimum temperature you would like for your trip?'))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [11]:
# filter dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
13,13,Sinazongwe,ZM,2021-12-04 16:32:29,-17.2614,27.4618,88.09,39,1,5.26
14,14,Mahebourg,MU,2021-12-04 16:32:29,-20.4081,57.7000,79.25,73,20,5.75
16,16,Saint-Philippe,RE,2021-12-04 16:32:29,-21.3585,55.7679,80.19,82,36,10.40
17,17,Rikitea,PF,2021-12-04 16:32:30,-23.1203,-134.9692,75.42,73,56,12.91
20,20,Puerto Escondido,MX,2021-12-04 16:32:30,15.8500,-97.0667,85.03,70,1,0.00
...,...,...,...,...,...,...,...,...,...,...
538,538,Merauke,ID,2021-12-04 16:46:03,-8.4667,140.3333,81.07,83,100,8.28
541,541,Tomatlan,MX,2021-12-04 16:46:04,19.9333,-105.2500,84.81,60,20,1.45
542,542,Canazas,PA,2021-12-04 16:46:05,8.3167,-81.2167,79.77,75,58,7.72
546,546,Niteroi,BR,2021-12-04 16:46:06,-22.8833,-43.1036,80.22,69,75,13.80


In [12]:
# check nulls in preferred_cities_df
preferred_cities_df.count()

City_ID       154
City          154
Country       154
Date          154
Lat           154
Lng           154
Max Temp      154
Humidity      154
Cloudiness    154
Wind Speed    154
dtype: int64

In [13]:
# create a copy of preferred_cities_df to store hotel names 
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp', 'Lat', "Lng"]].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
13,Sinazongwe,ZM,88.09,-17.2614,27.4618,
14,Mahebourg,MU,79.25,-20.4081,57.7000,
16,Saint-Philippe,RE,80.19,-21.3585,55.7679,
17,Rikitea,PF,75.42,-23.1203,-134.9692,
20,Puerto Escondido,MX,85.03,15.8500,-97.0667,
22,San Patricio,US,78.91,28.0170,-97.5169,
23,Sao Filipe,CV,77.09,14.8961,-24.4956,
32,Poum,NC,79.34,-20.2333,164.0167,
34,Manokwari,ID,78.48,-0.8667,134.0833,
36,Bambous Virieux,MU,79.21,-20.3428,57.7575,


In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError, AttributeError):
        print("Hotel not found... skipping.")
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
13,Sinazongwe,ZM,88.09,-17.2614,27.4618,Lakeview Lodge
14,Mahebourg,MU,79.25,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
16,Saint-Philippe,RE,80.19,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
17,Rikitea,PF,75.42,-23.1203,-134.9692,People ThankYou
20,Puerto Escondido,MX,85.03,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
22,San Patricio,US,78.91,28.0170,-97.5169,"Motel 6 Sinton, TX"
23,Sao Filipe,CV,77.09,14.8961,-24.4956,Hotel Xaguate
32,Poum,NC,79.34,-20.2333,164.0167,Gîte kajeon
34,Manokwari,ID,78.48,-0.8667,134.0833,Swiss Belhotel Manokwari
36,Bambous Virieux,MU,79.21,-20.3428,57.7575,Casa Tia Villa


In [15]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [17]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [18]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))